In [2]:
import pandas as pd
from hazm import Normalizer, word_tokenize, stopwords_list
from tqdm import tqdm  # <--- Changed from tqdm.auto to just tqdm

# Force text-mode progress bar for Pandas
tqdm.pandas(bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]')

# Load Data
df = pd.read_csv('../data/processed/clean_data.csv')

# Tools
normalizer = Normalizer()
stop_words = set(stopwords_list())

def preprocess_step_1(text):
    if not isinstance(text, str): return []
    try:
        # 1. Normalize
        text = normalizer.normalize(text)
        # 2. Tokenize
        tokens = word_tokenize(text)
        # 3. Stopword Removal
        clean_tokens = [t for t in tokens if t not in stop_words]
        return clean_tokens
    except:
        return []

print("Running Preprocessing ...")

df['clean_tokens_list'] = df['comment_text'].progress_apply(preprocess_step_1)

print("Process Complete.")
print(df[['comment_text', 'clean_tokens_list']].head(2))

Running Preprocessing ...


100%|██████████| 4548/4548 [25:38<00:00]

Process Complete.
                                        comment_text  \
0                                چیزی اضافه ایی نیست   
1  خوب درس نمیده اصلا، نمره ها رو خوب نمیده و اصل...   

                                   clean_tokens_list  
0                                       [اضافه, ایی]  
1  [درس, نمیده, اصلا, ،, نمره‌ها, نمیده, اصلا, تر...  


In [3]:
# ---------------------------------------------------------
#            Simple Rule-Based Sentiment
# ---------------------------------------------------------

# 1. Define Lexicons (Keywords)
# We add Persian keywords common in student feedback
positive_keywords = {
    'خوب', 'عالی', 'بهترین', 'راضی', 'مفید', 'مهربان', 
    'با‌سواد', 'محترم', 'قوی', 'منصف', '۲۰', '20', 'تشکر',
    'استاد نمونه', 'خوش برخورد', 'مسلط'
}

negative_keywords = {
    'بد', 'ضعیف', 'بی‌کیفیت', 'افتضاح', 'حیف', 'بی‌سواد', 
    'نامرد', 'سخت‌گیر', 'بی‌مسئولیت', 'خسته', 'الکی', 'نمی‌دهد',
    'عقده ای', 'پاس نمیکنه', 'میندازه', 'نمره نمیده'
}

# 2. Define the Rule-Based Function
def get_simple_sentiment(tokens):
    score = 0
    # Safety check: ensure we have a list
    if not isinstance(tokens, list): return 'Neutral', 0
    
    for token in tokens:
        if token in positive_keywords:
            score += 1
        elif token in negative_keywords:
            score -= 1
            
    # Determine Label
    if score > 0:
        label = 'Positive'
    elif score < 0:
        label = 'Negative'
    else:
        label = 'Neutral'
        
    return label, score

# 3. Apply to DataFrame

# Note: This runs very fast (seconds)
df[['simple_label', 'simple_score']] = df['clean_tokens_list'].progress_apply(
    lambda x: pd.Series(get_simple_sentiment(x))
)

print("Process Complete.")
print("Distribution of Simple Labels:")
print(df['simple_label'].value_counts())

100%|██████████| 4548/4548 [00:00<00:00]

Process Complete.
Distribution of Simple Labels:
simple_label
Neutral     3703
Positive     483
Negative     362
Name: count, dtype: int64


In [4]:
# ---------------------------------------------------------
#                 ParsBERT Sentiment
# ---------------------------------------------------------
import os
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

# 1. Configuration
HUB_MODEL_ID = "HooshvareLab/bert-fa-base-uncased-sentiment-digikala"
LOCAL_MODEL_PATH = "../models/parsbert_sentiment"

def load_sentiment_pipeline():
    # --- SAFETY: Create the folder if it doesn't exist ---
    if not os.path.exists(LOCAL_MODEL_PATH):
        os.makedirs(LOCAL_MODEL_PATH, exist_ok=True)
    # -----------------------------------------------------

    # Check if files already exist inside
    if os.listdir(LOCAL_MODEL_PATH):
        print(f"Found local model at '{LOCAL_MODEL_PATH}'. Loading from disk...")
        model_source = LOCAL_MODEL_PATH
    else:
        print(f"Local model not found. Downloading from Hugging Face (~450MB)...")
        model_source = HUB_MODEL_ID
    
    # Load Model & Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_source)
    model = AutoModelForSequenceClassification.from_pretrained(model_source)
    
    # Save to disk if we just downloaded it
    if model_source == HUB_MODEL_ID:
        print(f"Saving model to '{LOCAL_MODEL_PATH}'...")
        tokenizer.save_pretrained(LOCAL_MODEL_PATH)
        model.save_pretrained(LOCAL_MODEL_PATH)
        print("Model saved successfully.")

    return pipeline("text-classification", model=model, tokenizer=tokenizer)

# 2. Initialize
sentiment_pipeline = load_sentiment_pipeline()

# 3. Define Helper
def get_bert_sentiment(text):
    if not isinstance(text, str) or len(text.strip()) < 2:
        return "Neutral", 0.0
    
    # Truncate to 1000 chars (approx 200-300 words)
    safe_text = text[:1000]
    
    try:
        result = sentiment_pipeline(safe_text, truncation=True, max_length=512)[0]
        label = result['label']
        score = result['score']
        
        if label == 'recommended':
            return 'Positive', score
        elif label == 'not_recommended':
            return 'Negative', -score
        elif label == 'no_idea':
            return 'Neutral', 0
        else:
            return label, score
    except:
        return "Error", 0

# 4. Run Inference
tqdm.pandas(desc="Running ParsBERT")
print("Running AI on 4,500 rows ...")

df[['bert_label', 'bert_score']] = df['clean_comment_text'].progress_apply(
    lambda x: pd.Series(get_bert_sentiment(x))
)

print("Process Complete!")
display(df[['clean_comment_text', 'simple_label', 'bert_label']].head(10))

Found local model at '../models/parsbert_sentiment'. Loading from disk...


Device set to use cpu


Running AI on 4,500 rows ...


Running ParsBERT: 100%|██████████| 4548/4548 [06:15<00:00, 12.12it/s]

Process Complete!


,clean_comment_text,simple_label,bert_label
0,چیزی اضافه ایی نیست,Neutral,Negative
1,خوب درس نمیده اصلا، نمره ها رو خوب نمیده و اصل...,Neutral,Negative
2,در کل اگر که دنبال یک استاد با ادب با دانشجو م...,Neutral,Positive
3,سخت گیر و پربازده,Negative,Positive
4,چیزی اضافه ایی نیست,Neutral,Negative
5,دقت کنید هر جلسه مطالب را بخونید ارشد شب امتحا...,Neutral,Positive
6,چیزی اضافه ایی نیست,Neutral,Negative
7,اعصابی براتون نمیمونه چه نمره خوبی بگیرید چه ب...,Neutral,Neutral
8,استاد خوبی هستند فقط امتحان های سختی میگیرن که...,Neutral,Positive
9,سعی کنید سر کلاس خوب گوش بدید به مطالب,Neutral,Positive


In [5]:
# ---------------------------------------------------------
#                   FINAL COMPARISON
# ---------------------------------------------------------

# 1. View the Raw Comparison
print("Comparison of Methods:")
print(df[['clean_comment_text', 'simple_label', 'bert_label']].head(10))

# 2. Find the "Disagreements" (Where AI is smarter!)
# We filter for rows where the Simple Method failed (Neutral) but AI found meaning
mask_improved = (df['simple_label'] == 'Neutral') & (df['bert_label'] != 'Neutral')

print(f"\nInsight: The AI found sentiment in {mask_improved.sum()} rows that the Simple Method missed!")

# Show 5 examples where AI was smarter
print("\n--- Examples where AI was smarter ---")
display(df.loc[mask_improved, ['clean_comment_text', 'simple_label', 'bert_label']].head(5))

Comparison of Methods:
                                  clean_comment_text simple_label bert_label
0                                چیزی اضافه ایی نیست      Neutral   Negative
1  خوب درس نمیده اصلا، نمره ها رو خوب نمیده و اصل...      Neutral   Negative
2  در کل اگر که دنبال یک استاد با ادب با دانشجو م...      Neutral   Positive
3                                  سخت گیر و پربازده     Negative   Positive
4                                چیزی اضافه ایی نیست      Neutral   Negative
5  دقت کنید هر جلسه مطالب را بخونید ارشد شب امتحا...      Neutral   Positive
6                                چیزی اضافه ایی نیست      Neutral   Negative
7  اعصابی براتون نمیمونه چه نمره خوبی بگیرید چه ب...      Neutral    Neutral
8  استاد خوبی هستند فقط امتحان های سختی میگیرن که...      Neutral   Positive
9             سعی کنید سر کلاس خوب گوش بدید به مطالب      Neutral   Positive

Insight: The AI found sentiment in 2832 rows that the Simple Method missed!

--- Examples where AI was smarter ---


,clean_comment_text,simple_label,bert_label
0,چیزی اضافه ایی نیست,Neutral,Negative
1,خوب درس نمیده اصلا، نمره ها رو خوب نمیده و اصل...,Neutral,Negative
2,در کل اگر که دنبال یک استاد با ادب با دانشجو م...,Neutral,Positive
4,چیزی اضافه ایی نیست,Neutral,Negative
5,دقت کنید هر جلسه مطالب را بخونید ارشد شب امتحا...,Neutral,Positive


In [6]:
# ---------------------------------------------------------
# SAVE PHASE 3 RESULTS
# ---------------------------------------------------------

# Save to a new CSV so we don't overwrite the original clean data
output_path = '../data/processed/sentiment_data.csv'

print(f"Saving sentiment results to '{output_path}'...")
df.to_csv(output_path, index=False)

print("Data saved safely.")
print(f"Total Rows: {len(df)}")

Saving sentiment results to '../data/processed/sentiment_data.csv'...
Data saved safely.
Total Rows: 4548
